In [23]:
import pandas as pd
import matplotlib.pyplot as plt

In [27]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join('..', 'src')))
from db_conexion import establecer_conexion


In [3]:
conn, cursor = establecer_conexion()

# Consulta SQL para seleccionar los datos
query = "SELECT * FROM grammy_awards"

# Leer los datos en un DataFrame de pandas
grammy = pd.read_sql_query(query, conn)
#grammy
#print(df.isnull().sum())

Conexion exitosa a la base de datos


C:\Users\juanm\AppData\Local\Temp\ipykernel_20964\2051416096.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  grammy = pd.read_sql_query(query, conn)


In [4]:
# Agregar una columna id al principio del csv
grammy['id'] = range(1, len(grammy) + 1)

# Mover la columna 'id' al principio
cols = ['id'] + [col for col in grammy.columns if col != 'id']
grammy = grammy[cols]

# Reemplazar los valores nulos de Worker, por trabajador desconocido
grammy['workers'] = grammy['workers'].fillna("Unknown Worker")
print(grammy['workers'].isnull().sum())

# Reemplazar los valor nulos de nominee, por desconocido
grammy['nominee'] = grammy['nominee'].fillna("Unknown")
print(grammy['nominee'].isnull().sum())

grammy['img'] = grammy['img'].fillna("https://soundimaging.com/wp-content/uploads/2020/12/coming.jpeg")
print(grammy.isnull().sum())

0
0
id                 0
year               0
title              0
published_at       0
updated_at         0
category           0
nominee            0
artist          1840
workers            0
img                0
winner             0
dtype: int64


In [30]:
from API_conexion import client_id,client_secret

In [31]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# Credenciales de la API de Spotify (debes reemplazar estos valores por tus credenciales)
client_id = client_id
client_secret = client_secret

# Autenticación con la API de Spotify
auth_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(auth_manager=auth_manager)

# Función para buscar el artista en Spotify usando el nombre de la canción
def obtener_artista_de_cancion(nombre_cancion):
    try:
        # Realizar búsqueda en Spotify
        resultados = sp.search(q=f'track:{nombre_cancion}', type='track', limit=1)
        if resultados['tracks']['items']:
            # Retornar el nombre del artista
            return resultados['tracks']['items'][0]['artists'][0]['name']
        else:
            return None
    except Exception as e:
        print(f"Error buscando la canción {nombre_cancion}: {e}")
        return None

# Rellenar valores nulos en la columna 'artist' usando el nombre del artista extraído de Spotify
grammy['artist'] = grammy.apply(lambda row: obtener_artista_de_cancion(row['nominee']) if pd.isnull(row['artist']) else row['artist'], axis=1)


In [32]:
grammy['artist'] = grammy['artist'].fillna("Unknown Artist or Various Artists")


print(grammy.isnull().sum())

# Guardar el archivo CSV actualizado
grammy.to_csv('../data/grammys_updated.csv', index=False)

print("Archivo actualizado y guardado.")



id               0
year             0
title            0
published_at     0
updated_at       0
category         0
nominee          0
artist           0
workers          0
img              0
winner           0
nominee_clean    0
dtype: int64
Archivo actualizado y guardado.
